## Stadtentwicklung

Open in Colab: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/febse/opt2025-de/blob/main/IP-Urban-Planning.ipynb)

Angenommen, Sofia steht vor einem lokalen Haushaltsdefizit und der Stadtrat überlegt, wie die Grundsteuereinnahmen durch die Sanierung städtischer Grundstücke erhöht werden können. Das Projekt besteht aus zwei Teilen: der Beseitigung vernachlässigter und beschädigter Gebäude und dem Bau neuer Wohnungen.

1. Die Gemeinde besitzt derzeit 300 baufällige Gebäude, die abgerissen werden könnten. Jedes dieser Gebäude macht 1,000 Quadratmeter frei und kostet 3,000 EUR pro Gebäude. 15 Prozent der befreiten Fläche sind für Straßen, Gehwege und Freiflächen vorgesehen.
2. Auf den geräumten Grundstücken kann die Gemeinde vier Arten von neuen Wohngebäuden errichten: Einfamilienhäuser (300 Quadratmeter), Zweifamilienhäuser (500 Quadratmeter), Dreifamilienhäuser (700 Quadratmeter) und Vierfamilienhäuser (900 Quadratmeter). Die geschätzten Steuereinnahmen belaufen sich auf EUR 1,000, EUR 1,700,  EUR 2,400 bzw.  EUR 2,800 pro Jahr.
3. Mindestens 20 Prozent der Neubauten müssen Einfamilienhäuser sein, Zweifamilienhäuser müssen mindestens 20 Prozent ausmachen, und Drei- und Vierfamilienhäuser müssen (zusammen) mindestens ein Viertel aller Neubauten ausmachen.
4. Die Baukosten für neue Häuser betragen 50,000 EUR, 70,000 EUR, 130,000 EUR bzw. 160,000 EUR.
5. Die Gemeinde beabsichtigt, das Projekt durch ein Bankdarlehen zu finanzieren, das 15 Millionen EUR nicht überschreiten darf.

Wie viele Häuser jedes Typs sollte die Gemeinde bauen, um die höchstmöglichen Steuereinnahmen zu erzielen?

Der Datensatz `homes` enthält die Daten für die vier Arten von Wohngebäuden:

- `cost`: Die Baukosten für ein Haus dieses Typs.
- `tax`: Die geschätzten Steuereinnahmen pro Jahr für ein Haus dieses Typs.
- `area`: Die Fläche, die ein Haus dieses Typs einnimmt.

In [2]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

homes = pd.DataFrame({
    'type': ['single', 'double', 'triple', 'quad'],
    'cost': [50000, 70000, 130000, 160000],
    "tax": [1000, 1700, 2400, 2800],
    'area': [300, 500, 700, 900],
}).set_index('type')

homes

,cost,tax,area
type,,,
single,50000,1000,300
double,70000,1700,500
triple,130000,2400,700
quad,160000,2800,900


In [21]:
# Unser bisherige Ansatz für die Umsetzung des Modells war es, die Entscheidungsvariablen
# einzeln zu definieren.

model1 = gp.Model('homes-1')

x_single = model1.addVar(vtype=GRB.INTEGER, name='x_single')
x_double = model1.addVar(vtype=GRB.INTEGER, name='x_double')
x_triple = model1.addVar(vtype=GRB.INTEGER, name='x_triple')
x_quad = model1.addVar(vtype=GRB.INTEGER, name='x_quad')

# Zielfunktion

model1.setObjective(1000 * x_single + 1700 * x_double + 2400 * x_triple + 2800 * x_quad, GRB.MAXIMIZE) 

model1.write('homes-1.lp')

with open('homes-1.lp', 'r') as f:
    print(f.read())

\ Model homes-1
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  1000 x_single + 1700 x_double + 2400 x_triple + 2800 x_quad
Subject To
Bounds
Generals
 x_single x_double x_triple x_quad
End



In [11]:
# Der Index
homes.index

Index(['single', 'double', 'triple', 'quad'], dtype='object', name='type')

In [26]:
model2 = gp.Model('Beispiel')

# Erstellt eine Variable für jeden Wert des Index (für jeden Haustyp in diesem Fall)
x = model2.addVars(homes.index, vtype=GRB.INTEGER, name='x')
model2.update()

x

{'single': <gurobi.Var x[single]>,
 'double': <gurobi.Var x[double]>,
 'triple': <gurobi.Var x[triple]>,
 'quad': <gurobi.Var x[quad]>}

In [29]:
model2.setObjective(x.prod(homes['tax'].to_dict()), GRB.MAXIMIZE)

model2.write('homes-2.lp')

with open('homes-2.lp', 'r') as f:
    print(f.read())


\ Model Beispiel
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  1000 x[single] + 1700 x[double] + 2400 x[triple] + 2800 x[quad]
Subject To
Bounds
Generals
 x[single] x[double] x[triple] x[quad]
End



In [4]:
# Zugriff auf eine Variable
x['single']

<gurobi.Var x[single]>

In [5]:
x['quad']

<gurobi.Var x[quad]>

In [13]:
# Eine Linearkombination von zwei Variablen

2 * x['single'] + 3 * x['quad']

<gurobi.LinExpr: 2.0 x[single] + 3.0 x[quad]>

In [ ]:
# Die Summe aller Variablen

x.sum()

<gurobi.LinExpr: x[single] + x[double] + x[triple] + x[quad]>

In [20]:
# Die Werte einer Spalte des DataFrames als Dictionary

homes['area'].to_dict()

{'single': 300, 'double': 500, 'triple': 700, 'quad': 900}

In [19]:
homes['area'].to_dict()['single']

300

In [ ]:
# Eine lineare Ungleichung mit Koeffizienten aus einem Dictionary

x.prod(homes['area'].to_dict()) <= 2000

<gurobi.TempConstr: 300.0 x[single] + 500.0 x[double] + 700.0 x[triple] + 900.0 x[quad] <= 2000>

In [18]:
m = gp.Model('homes')

x = m.addVars(homes.index, vtype=GRB.INTEGER, name='x')
x_d = m.addVar(vtype=GRB.INTEGER, name='demolished')

# Die Zielfunktion

# m.setObjective(..., GRB.MAXIMIZE)

# Die Einschränkungen

## Kosten
# m.addConstr(x.prod(homes['cost'].to_dict()) + ... <= 15e6, 'costs')

## Fläche
# m.addConstr(... <= ..., 'area')

## Anzahl der Einfamilienhäuser
m.addConstr(x['single'] >= 0.2 * x.sum(), 'single homes')

## Anzahl der Zweifamilienhäuser
# m.addConstr(..., 'double homes')

## Anzahl der Drei- und Vierfamilienhäuser
# m.addConstr(..., 'triple and quad homes')

# Die Lösung

# m.optimize()

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
# m.write('homes.lp')

# with open('homes.lp', 'r') as f:
#     print(f.read())

\ Model homes
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  1000 x[single] + 1700 x[double] + 2400 x[triple] + 2800 x[quad]
Subject To
 costs: 50000 x[single] + 70000 x[double] + 130000 x[triple]
   + 160000 x[quad] + 3000 x_d <= 1.5e+07
 area: 300 x[single] + 500 x[double] + 700 x[triple] + 900 x[quad]
   - 999.85 x_d <= 0
 single_homes: 0.8 x[single] - 0.2 x[double] - 0.2 x[triple] - 0.2 x[quad]
   >= 0
 double_homes: - 0.2 x[single] + 0.8 x[double] - 0.2 x[triple]
   - 0.2 x[quad] >= 0
 triple_and_quad_homes: - 0.25 x[single] - 0.25 x[double] + 0.75 x[triple]
   + 0.75 x[quad] = 0
Bounds
Generals
 x[single] x[double] x[triple] x[quad] x_d
End



In [ ]:
# Die Lösung als DataFrame

# vars_df = pd.DataFrame(
#     [(v.varName, v.x) for v in m.getVars()],
#     columns=['variable', 'value']
# )

# vars_df

,variable,value
0,x[single],36.0
1,x[double],99.0
2,x[triple],41.0
3,x[quad],4.0
4,x_d,93.0


In [ ]:
# Die Einschränkungen as DataFrame

# constrs_df = pd.DataFrame(
#     [(c.constrName, c.slack) for c in m.getConstrs()],
#     columns=['constraint', 'slack']
# )

# constrs_df

,constraint,slack
0,costs,0.000000
1,area,0.000000
2,single homes,0.000000
3,double homes,-63.613055
4,triple and quad homes,0.000000
